In [0]:
!pip install -q deeppavlov
!python -m deeppavlov install squad_bert
!python -m pip install pybind11==2.2.3 git+https://github.com/deepmipt/fastText.git#egg=fastText==0.8.22
#!pip install --upgrade sklearn

ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 0.25.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
2020-05-17 12:06:17.241 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_bert' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/squad/squad_bert.json'
     |████████████████████████████████| 110.5MB 98kB/s 
     |████████████████████████████████| 512kB 56.6MB/s 
     |████████████████████████████████| 3.8MB 51.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=dfcbb1913c49d8f6deabf15b3c5f9c8398ce6bac2a99b0b9bb1ffc7922c1a87c
  Stored in dir

In [0]:
!pip install nltk
!pip install langdetect

     |████████████████████████████████| 983kB 11.0MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=0d28b480f58baef3c35d6ca4a1fc5feb0e3c2dc9ccea177ab27bfb165a4c88e4
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [0]:
!pip install --upgrade pandas

Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.0.3)
     |████████████████████████████████| 20.2MB 1.2MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [0]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm_notebook as tqdm
import random
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from google.colab import drive
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.naive_bayes import *
import warnings
import pickle as pkl
#from langdetect import detect
warnings.filterwarnings('ignore')

In [0]:
drive.mount('/content/drive/')

In [0]:
!ls -l "/content/drive/My Drive"

total 1486137
-rw------- 1 root root     130112 Dec 18 15:58  Autos_Quiz.ipynb
-rw------- 1 root root   20835209 Dec 18 15:25  autos_short.csv
drwx------ 2 root root       4096 Nov 14  2018 'Colab Notebooks'
drwx------ 2 root root       4096 Jul  8  2019  danya_examples
-rw------- 1 root root  392335360 Jan  5 23:07  kth-tips2-a.tar
drwx------ 2 root root       4096 Jan  9 17:59 'kth-tips2-a.tar (Unzipped Files)'
-rw------- 1 root root 1108119164 Jan  9 22:16 'Neural nets.zip'
drwx------ 2 root root       4096 Nov 24 08:53  passports
drwx------ 2 root root       4096 Oct 13  2019  study
drwx------ 2 root root       4096 Sep 17  2017  TelegramVkClientFiles
drwx------ 2 root root       4096 Mar  4 20:02 'TUM Application'
-rw------- 1 root root       2960 Apr 21 14:37 'Untitled Diagram.drawio'
drwx------ 2 root root       4096 Sep 30  2019  Work
drwx------ 2 root root       4096 May  4  2019 'Дмитриев Антон (БПМ-16-1)'
-rw------- 1 root root     237056 Apr  2  2018 'Задача о фальшивой мо

In [0]:
train_frame = pd.read_excel('/content/drive/My Drive/Work/OnlineNewsScoring/data/dataset_news_2018_2019_marked_full.xlsx')
train_frame = train_frame[[i for i in train_frame.columns if 'Unnamed' not in i]]
#train_frame = train_frame[['title', 'description', 'cat', 'sentiment']]
train_frame = train_frame.fillna('')
train_frame = train_frame[np.logical_or(train_frame['title'] != '', train_frame['description'] != '')]
train_frame['cat'] = train_frame['cat'].astype('int')
train_frame['sentiment'] = train_frame['sentiment'].astype('int')
train_frame['sentiment'] = train_frame['sentiment'].replace({9: 0})

In [0]:
train_frame.head()

,u_id,date_time,provider,title,description,link,pubdate,numfield,cat,sentiment
0,4573258,2018-12-01 13:00:07,newrucom_economy,&quot;Роснефть&quot; остановила продажу топлив...,Крупнейший торговец топливом неожиданно отказа...,https://www.newsru.com/finance/15nov2018/rosne...,15.11.2018 16:26,4,5,0
1,4573275,2018-12-01 13:00:07,newrucom_economy,Fitch усомнилось в перспективах &quot;Нафтогаз...,Fitch присвоило еврооблигациям &quot;Нафтогаза...,https://www.newsru.com/finance/06nov2018/not_c...,06.11.2018 12:10,4,5,-1
2,4573279,2018-12-01 13:00:07,newrucom_economy,&quot;Роснефть&quot; отказалась от иска к парт...,Представитель &quot;Роснефти&quot; заявил об о...,https://www.newsru.com/finance/02nov2018/net_s...,02.11.2018 10:05,4,5,0
3,4573336,2018-12-01 13:00:12,inosmiru_economy,Neue Zrcher Zeitung (Швейцария): Газпрому прид...,Владимир Путин и Реджеп Тайип Эрдоган пребываю...,https://inosmi.ru/economic/20181121/244049639....,21.11.2018 00:27,4,2,1
4,4573494,2018-12-01 13:00:27,kpru_economy,«Роснефть»: Ажиотаж вокруг дефицита топлива ис...,Компания призывает навести порядок на бирже,http://www.kp.ru/daily/26909/3955475/,19.11.2018 23:22,4,5,0


In [0]:
train_frame['provider'].unique()

array(['newrucom_economy', 'inosmiru_economy', 'kpru_economy',
       'svobodaorg_economy', 'cnews_agenda', 'vsesmiru_finance',
       'yandexru_business', 'regnumru_polit', 'federalpressru_common',
       'kommersantru_business', '1primeru_economy', 'rgru_common',
       'tassru_common', 'pravo_gov_common', 'izvestiaru_common',
       'vestifinance_common', 'cnews_integration', 'ngru_common',
       'interfaxru_common', 'vedomostiru_common', 'eadailycom_common',
       'cnews_biz', 'tvrainru_common', 'russianrtcom_common',
       'regnumru_economy', 'novaya_gazeta', 'yandexru_polit',
       'gazetaru_business', 'vsesmiru_business', 'mailru_common',
       'nakanuneru_common', 'regnumru_russia', 'vzru_common',
       'yandexru_common', 'gazetaru_common', 'expertru_common',
       'lentaru_common', 'kpru_common', 'kommersantru_main',
       'svobodaorg_common', 'vestiru_common', 'newrucom_common',
       'ramblerru_polit', 'mailru_main', 'cnews_all', 'cnews_top',
       'infoxru_busines

In [0]:
def condition(label):
    conds = []
    conds.append('o' != label.lower())
    return np.any(conds)
  
def condition_str(word):
    conds = []
    conds.append(word != '')
    conds.append('&quot;' not in word)
    conds.append('"' != word)
    conds.append("'" != word)
    return np.all(conds)
  
def create_feature_col(frame, use_ner=False):
    frame_copy = frame.sample(frac=1).reset_index(drop=True)
    titles = []
    for title in tqdm(frame_copy['title'] + ' ' + frame_copy['description']):
        title = ' '.join([i for i in title.split(' ') if '$' not in i])
        title = title.replace('&quot;', '"')
        if use_ner:
            mapping = ner_model([title])
            sent = np.array(mapping[0][0])
            ner = mapping[1][0]
            new_sent_ind = []
            for i, label in enumerate(ner):
                if not condition(label):
                    new_sent_ind.append(i)
            string = ' '.join(sent[new_sent_ind])
        else:
            string = title
        string = string.lower()
        string = ' '.join([i for i in string.split() if condition_str(i)])
        string = re.sub(r'&amp;laquo;', '', string)
        string = re.sub(r'<.*>', '', string)
        string = re.sub(r'[\W]', ' ', string)
        string = re.sub(r' +', ' ', string)
        titles.append(string.strip())
    frame_copy['title_proc'] = titles
    return frame_copy
  
class_map = {
    1: "Убытки",
    2: "Модернизация",
    3: "Инвестиции",
    4: "Санкции",
    5: "Другое",
    6: "Постановления",
}

def class_distribution(frame, class_col, mapp=None):
    for i in sorted(frame[class_col].unique()):
        count = len(frame[frame[class_col] == i])
        proportion = round(len(frame[frame[class_col] == i]) / len(frame), 2)
        if mapp is not None:
            print('{} -> {} ({} of population)'.format(mapp[i], count, proportion))
        else:
            print('{} -> {} ({} of population)'.format(i, count, proportion))
    print(len(frame))

In [0]:
class_distribution(train_frame, 'cat', class_map)

Убытки -> 36 (0.02 of population)
Модернизация -> 100 (0.06 of population)
Инвестиции -> 162 (0.09 of population)
Санкции -> 77 (0.04 of population)
Другое -> 1346 (0.75 of population)
Постановления -> 83 (0.05 of population)
1804


In [0]:
frame = train_frame.copy()

In [0]:
indexes = random.sample(list(frame[frame.cat == 5].index), 500)
frame = frame.drop(indexes, axis=0).reset_index(drop=True)
print(len(frame))

1304


In [0]:
class_distribution(frame, 'cat', class_map)

Убытки -> 36 (0.03 of population)
Модернизация -> 100 (0.08 of population)
Инвестиции -> 162 (0.12 of population)
Санкции -> 77 (0.06 of population)
Другое -> 846 (0.65 of population)
Постановления -> 83 (0.06 of population)
1304


In [0]:
class_distribution(train_frame, 'sentiment')

-1 -> 363 (0.2 of population)
0 -> 937 (0.52 of population)
1 -> 504 (0.28 of population)
1804


In [0]:
frame = create_feature_col(frame)

In [0]:
vect = TfidfVectorizer(min_df=10, max_df=500)
#vect = CountVectorizer(min_df=10, max_df=500)
data = vect.fit_transform(frame['title_proc'])

In [0]:
data.shape

(804, 467)

### SVM Experiments

In [0]:
from sklearn.svm import SVC

class ModelCat:
  
    def __init__(self, kernel, C):
        self.vect = TfidfVectorizer(min_df=10, max_df=500)
        self.model = SVC(C, kernel=kernel)
    
    def fit(self, X, y):
        data = self.vect.fit_transform(X)
        self.model.fit(data.toarray(), y)
        
    def predict(self, X):
        data = self.vect.transform(X)
        preds = self.model.predict(data.toarray())
        for i, x in enumerate(X):
            if x.split(' ')[0].lower() == 'приказ':
                preds[i] = 5
                continue
            if x.split(' ')[0].lower() == 'постановление':
                preds[i] = 6
                continue
            for part in x.split(' '):
                if 'инвест' in part.lower():
                    preds[i] = 3
                    break
                if 'санкц' in part.lower() and 'несанкц' not in part.lower():
                    preds[i] = 4
                    break
                if 'развивать' == part.lower():
                    preds[i] = 2
                    break
                if part.lower() in ['убытки', 'потери', 'штраф']:
                    preds[i] = 1
        return preds

In [0]:
from itertools import product

def cross_val_svm(frame, target_col, prior='f1', n_first=5):
    results = []

    cs = np.linspace(0.001, 3, 100)
    kernels = ['linear', 'poly', 'rbf', 'sigmoid']

    for c, kernel in product(cs, kernels):
        kf = StratifiedKFold(n_splits=5, shuffle=True)
        splits = kf.split(data, frame[target_col])
        acc_scores = []
        f1s = []
        for train_index, test_index in splits:
          
            train_data = frame['title_proc'][train_index]
            train_lab = frame[target_col][train_index]
            test_data = frame['title_proc'][test_index]
            test_lab = frame[target_col][test_index]
            
            lr = ModelCat(C=c, kernel=kernel)
                
            lr.fit(train_data, train_lab)
            preds = lr.predict(test_data)
            
            acc_scores.append(sum(preds == test_lab) / len(preds))
            f1s.append(f1_score(test_lab, preds, average='weighted'))
            #roc_auc_score(test_lab, preds, average='weighted')
            
        results.append((c, kernel,
                        sum(acc_scores) / len(acc_scores), 
                        sum(f1s) / len(f1s)))
    if prior == 'acc':
        return sorted(results, key=lambda x: -x[2])[:n_first]
    elif prior == 'f1':
        return sorted(results, key=lambda x: -x[3])[:n_first]

In [0]:
cross_val_svm(frame, 'cat', 'f1')

[(1.7276969696969697, 'rbf', 0.7413121118012422, 0.729458610353277),
 (2.060919191919192, 'rbf', 0.7425465838509318, 0.7288701249283507),
 (1.8185757575757575, 'rbf', 0.7400543478260869, 0.7271441657704683),
 (2.454727272727273, 'rbf', 0.7375698757763975, 0.724617325839977),
 (2.1820909090909093, 'rbf', 0.7388664596273291, 0.7237856624796171)]

### SVM Experiments End

### RF Experiments

In [0]:
from sklearn.ensemble import RandomForestClassifier

class ModelCat:
  
    def __init__(self, n, criterion, max_depth):
        self.vect = TfidfVectorizer(min_df=10, max_df=500)
        self.model = RandomForestClassifier(n, criterion, max_depth)
    
    def fit(self, X, y):
        data = self.vect.fit_transform(X)
        self.model.fit(data.toarray(), y)
        
    def predict(self, X):
        data = self.vect.transform(X)
        preds = self.model.predict(data.toarray())
        for i, x in enumerate(X):
            if x.split(' ')[0].lower() == 'приказ':
                preds[i] = 5
                continue
            if x.split(' ')[0].lower() == 'постановление':
                preds[i] = 6
                continue
            for part in x.split(' '):
                if 'инвест' in part.lower():
                    preds[i] = 3
                    break
                if 'санкц' in part.lower() and 'несанкц' not in part.lower():
                    preds[i] = 4
                    break
                if 'развивать' == part.lower():
                    preds[i] = 2
                    break
                if part.lower() in ['убытки', 'потери', 'штраф']:
                    preds[i] = 1
        return preds

In [0]:
from itertools import product

def cross_val_RF(frame, target_col, prior='f1', n_first=5):
    results = []

    n_est = np.linspace(100, 5000, 100)
    criterions = ['gini', 'entropy']
    max_depths = np.linspace(20, 200, 100)

    for n, c, md in product(n_est, criterions, max_depths):
        kf = StratifiedKFold(n_splits=5, shuffle=True)
        splits = kf.split(data, frame[target_col])
        acc_scores = []
        f1s = []
        for train_index, test_index in splits:
          
            train_data = frame['title_proc'][train_index]
            train_lab = frame[target_col][train_index]
            test_data = frame['title_proc'][test_index]
            test_lab = frame[target_col][test_index]
            
            lr = ModelCat(int(n), c, int(md))
                
            lr.fit(train_data, train_lab)
            preds = lr.predict(test_data)
            
            acc_scores.append(sum(preds == test_lab) / len(preds))
            f1s.append(f1_score(test_lab, preds, average='weighted'))
            #roc_auc_score(test_lab, preds, average='weighted')
            
        results.append((c, n, c, md,
                        sum(acc_scores) / len(acc_scores), 
                        sum(f1s) / len(f1s)))
    if prior == 'acc':
        return sorted(results, key=lambda x: -x[4])[:n_first]
    elif prior == 'f1':
        return sorted(results, key=lambda x: -x[5])[:n_first]

In [0]:
cross_val_RF(frame, 'cat', 'f1')

### RF Experiments End

In [0]:
class ModelCat:
  
    def __init__(self, alpha):
        self.vect = TfidfVectorizer(min_df=10, max_df=500)
        self.model = MultinomialNB(alpha)
    
    def fit(self, X, y):
        data = self.vect.fit_transform(X)
        self.model.fit(data.toarray(), y)
        
    def predict(self, X):
        data = self.vect.transform(X)
        preds = self.model.predict(data.toarray())
        for i, x in enumerate(X):
            if x.split(' ')[0].lower() == 'приказ':
                preds[i] = 5
                continue
            if x.split(' ')[0].lower() == 'постановление':
                preds[i] = 6
                continue
            for part in x.split(' '):
                if 'инвест' in part.lower():
                    preds[i] = 3
                    break
                if 'санкц' in part.lower() and 'несанкц' not in part.lower():
                    preds[i] = 4
                    break
                if 'развивать' == part.lower():
                    preds[i] = 2
                    break
                if part.lower() in ['убытки', 'потери', 'штраф']:
                    preds[i] = 1
        return preds

In [0]:
class ModelSent:
    
    def __init__(self, alpha):
        self.vect = TfidfVectorizer(min_df=10, max_df=500)
        self.model = MultinomialNB(alpha)
    
    def fit(self, X, y):
        data = self.vect.fit_transform(X)
        self.model.fit(data, y)
        
    def predict(self, X):
        data = self.vect.transform(X)
        preds = self.model.predict(data)
        for i, x in enumerate(X):
            for part in x.split(' '):
                if 'обсуд' in part:
                    preds[i] = 0
                    break
        return preds

In [0]:
def cross_val(frame, target_col, prior='acc', n_first=100):
    results = []
    for c in np.linspace(0.001, 3, 100):#, base=11):
        kf = StratifiedKFold(n_splits=5, shuffle=True)
        splits = kf.split(data, frame[target_col])
        acc_scores = []
        f1s = []
        for train_index, test_index in splits:
          
            train_data = frame['title_proc'][train_index]
            train_lab = frame[target_col][train_index]
            test_data = frame['title_proc'][test_index]
            test_lab = frame[target_col][test_index]
            
            if target_col == 'cat':
                lr = ModelCat(alpha=c)
            elif target_col == 'sentiment':
                lr = ModelSent(alpha=c)
                
            lr.fit(train_data, train_lab)
            preds = lr.predict(test_data)
            
            acc_scores.append(sum(preds == test_lab) / len(preds))
            f1s.append(f1_score(test_lab, preds, average='weighted'))
            #roc_auc_score(test_lab, preds, average='weighted')
            
        results.append((c, 
                        sum(acc_scores) / len(acc_scores), 
                        sum(f1s) / len(f1s)))
    if prior == 'acc':
        return sorted(results, key=lambda x: -x[1])[:n_first]
    elif prior == 'f1':
        return sorted(results, key=lambda x: -x[2])[:n_first]

In [0]:
cross_val(frame, 'cat', 'f1', 5)

[(0.031292929292929296, 0.7730002947244327, 0.7571806125037798),
 (0.21305050505050507, 0.779112879457707, 0.75693986035909),
 (0.18275757575757579, 0.7745358090185677, 0.7564822991114487),
 (0.30392929292929294, 0.7806660772178013, 0.7562469678786607),
 (0.06158585858585859, 0.7699351606248158, 0.7535523943292451)]

In [0]:
cross_val(frame, 'sentiment', 'f1', 5)

[(0.15246464646464647, 0.6809961685823754, 0.6842919016805327),
 (0.031292929292929296, 0.6755968169761274, 0.6788348147431514),
 (0.2736363636363637, 0.6748629531388153, 0.6775067724096162),
 (0.7280303030303031, 0.6733127026230475, 0.6761847068821223),
 (0.001, 0.6725316828765104, 0.6757180376256702)]

In [0]:
train_X = frame['title_proc'][:-200]
train_y = frame['sentiment'][:-200]
test_X = frame['title_proc'][-200:]
test_y = frame['sentiment'][-200:]

model = ModelSent(0.15246)
model.fit(train_X, train_y)
labels = model.predict(test_X)

for title, label in zip(frame['title'][-200:], labels):
    print(title.replace('&quot;', ''))
    print(label)
    print('______________________________-')

«РИА Новости»: «Газпром» собирается разместить 10-летние облигации в иенах
0
______________________________-
Выход из акций "Газпрома" продолжится
1
______________________________-
Газпром направит в 2019 году 53 млрд рублей на Северный поток-2
1
______________________________-
Приказ Региональной службы по тарифам Республики Калмыкия от 24.12.2018 № 104-п/тпг
 "Об утверждении стандартизированных тарифных ставок для определения платы за технологическое присоединение газоиспользующего оборудования заявителя к сетям газораспределения АО "Газпром газораспределение Элиста" на 2019 год"
0
______________________________-
«Роснефть» вышла на берег Ливана // Она взяла в управление нефтепродуктовый терминал
1
______________________________-
Приказ Региональной службы по тарифам Республики Калмыкия от 24.12.2018 № 102-п/г
 "Об утверждении розничной цены на природный газ, реализуемый АО "Газпром газораспределение Элиста" населению для бытовых нужд, а также жилищно-эксплуатационным организациям, о

In [0]:
mod = ModelSent(0.001)
import pickle as pkl
mod.fit(train_frame['title_proc'], train_frame['cat'])
pkl.dumps(mod, protocol=4)

In [0]:
pkl.compatible_formats

In [0]:
path_dir = '/content/drive/My Drive/OnlineNewsScoring/data/'
filenames = ['20170301.csv', '20170601.csv', '20170901.csv', '20171201.csv']
comp_dict = ['газпром', 'роснефт', 'лукойл', 'татнефт', 'новатэк']
cat_map = {
    1: 'Убытки',
    2: 'Модернизация',
    3: 'Инвестиции',
    4: 'Санкции',
    5: 'Другое',
    6: 'Постановления'
}

In [0]:
frame = pd.read_csv(path_dir + 'neftegaz_full.csv')
frame = frame.fillna('')
frame.head()

FileNotFoundError: ignored

In [0]:
frames = []
for name in filenames:
    frame = pd.read_csv(path_dir + name)
    frame = frame[['u_id', 'title', 'description']]
    frame = frame.fillna('')
    frame = frame[np.logical_or(frame['title'] != '', frame['description'] != '')]
    indexes = []
    for i, title in tqdm(enumerate(frame['title'] + ' ' + frame['description'])):
        for comp in comp_dict:
            if comp in title.lower():
                indexes.append(True)
                break
        if len(indexes) == i:
            indexes.append(False)
    assert len(indexes) == len(frame)
    print(sum(indexes))
    frame = frame[indexes]
    frames.append(frame)
frame = pd.concat(frames, axis=0)
frame = frame.drop_duplicates('u_id')
frame = frame.reset_index(drop=True)
frame = frame.fillna('')
print(len(frame))
frame.head()

NameError: ignored

In [0]:
frame.to_csv(path_dir + 'neftegaz_full.csv', index=False)

In [0]:
train_frame = create_feature_col(train_frame)

In [0]:
frame = create_feature_col(frame)
#arr = []
#for i in tqdm(frame['title'] + ' ' + frame['description']):
#    s = re.sub(r'[\W]', '', i).lower()
#    l = len(s)
#    if len(re.sub(r'[A-Za-z]', '', s).strip()) < l * 0.3:
#        arr.append(False)
#    else:
#        arr.append(True)
#frame = frame[arr]

In [0]:
model = ModelCat(alpha=0.0312)

In [0]:
model.fit(train_frame['title_proc'], train_frame['cat'])

In [0]:
preds = model.predict(frame['title_proc'])

In [0]:
set(preds)

{1, 2, 3, 4, 5, 6}

In [0]:
frame['cat'] = preds

In [0]:
for i, row in frame.head(20).iterrows():
    print(row['title'])

&quot;Газпромнефть-СМ&quot; в 2016 г. увеличит выпуск продукции на 4%, до 540 тыс. т
Экономика: Киев готовится начать новую газовую войну
СМИ: часть денег &quot;Роснефти&quot; Улюкаев должен был раздать подчиненным
"Газпром-Югра" в гостях обыграл "Прогресс" в первом матче 14-го тура ЧР
Арбитражный суд рассмотрит кассационную жалобу &quot;Роснефти&quot; к РБК
Republic: «Роснефть» и РБК заключили мировое соглашение
Киевский суд удовлетворил иск к «Газпрому» на $6,8 млрд
Предправления «Газпромбанка» Акимов и министр Новак покинули совет директоров «Роснефти»
Делегация &quot;Газпрома&quot; может в середине декабря посетить Японию
«Газпром» заключил новый договор со «Стройтранснефтегазом» Геннадия Тимченко
«Нафтогаз» сообщил о подаче «Газпромом» заявки на увеличение транзита газа через Украину
ФАС предостерегла Алекперова от высказываний о ценах на бензин
Прогноз результатов "Новатэка" за I квартал: чистая прибыль снизится на 48%, - Алексей Кокин,аналитик корпорации "Уралсиб"
Манчини встрет

In [0]:
for i, row in frame.sample(frac=1).head(20).iterrows():
    print(row['title'])
    print(row['description'])
    print(cat_map[row['cat']])
    print('_________________________________________')

&quot;Роснефть&quot; в 2016 году сократила чистую прибыль вдвое
Чистая прибыль &quot;Роснефти&quot; по МСФО за 2016 год составила 181 млрд рублей, что вдвое меньше показателя предыдущего года, говорится в сообщении компании.
Другое
_________________________________________
Керосин взлетает // Авиакомпании заправляют баки
Восстановление пассажиропотока российских авиакомпаний приводит к постепенному росту потребления авиакеросина. По данным &quot;Газпромнефть-Аэро&quot;, в первом квартале спрос вырос на 4%, достигнув уровня начала 2015 года. При этом, по мнению компании, на рынке существует &quot;небольшой профицит&quot; керосина, сбалансировать который получается за счет поставок за рубеж.
Другое
_________________________________________
«Газпром» и «Нафтогаз» уверены, что выиграли в Стокгольме
«Нафтогаз» заявил, что формула цены пересмотрена в его пользу, а «Газпром» считает, что решение положительно для него
Другое
_________________________________________
Доля "Газпрома" на рынке ЕС

In [0]:
mistakes = [3]

In [0]:
frame[frame['cat'] == 3].sample(frac=1)[:20]

,title,description,title_proc,cat
12325,"Штраф в размере 2,3 млрд долларов, наложенный ...","Суд постановил, что крупнейший в России частны...",штраф в размере 2 3 млрд долларов наложенный н...,3
19743,UCP Ильи Щербовича продал акции «Транснефти» ф...,Фонд UCP Ильи Щербовича продал акции «Транснеф...,ucp ильи щербовича продал акции транснефти фин...,3
12291,«Нафтогаз» признал угрозу утраты Украиной тран...,"Европа останется один на один с «Газпромом», е...",нафтогаз признал угрозу утраты украиной транзи...,3
8542,&quot;Роснефть&quot; докапитализировала &quot;...,Сделка была одобрена советом директоров &quot;...,роснефть докапитализировала дочку рн иностранн...,3
1285,"""Газпром нефть"" инвестирует 15 млрд рублей в с...","<p>Проект, реализуемый «Газпром нефтью», получ...",газпром нефть инвестирует 15 млрд рублей в соз...,3
7492,Заголовки утра. 13 сентября,«Коммерсантъ» под заголовком «Лечение на вычит...,заголовки утра 13 сентября коммерсантъ под заг...,3
20795,Путин прибыл на судно-трубоукладчик для участи...,Президент России Владимир Путин в пятницу на в...,путин прибыл на судно трубоукладчик для участи...,3
19940,Китайская Beiging Gas Group решила приобрести ...,Китайская компания Beijing Gas Group направила...,китайская beiging gas group решила приобрести ...,3
9740,«Роснефть» за пять лет увеличит инвестиции в о...,Нефтяная компания «Роснефть» в ближайшие пять ...,роснефть за пять лет увеличит инвестиции в осв...,3
4540,«Газпром» вложит больше // Компания увеличила ...,Инвестпрограмма «Газпрома» на 2017 год после ч...,газпром вложит больше компания увеличила объем...,3


In [0]:
class_distribution(frame, 'cat')

1 -> 142 (0.01 of population)
2 -> 252 (0.01 of population)
3 -> 1990 (0.1 of population)
4 -> 815 (0.04 of population)
5 -> 17703 (0.85 of population)
6 -> 14 (0.0 of population)
20916


In [0]:
frame.loc[16042]['description']

'<img vspace="10px" hspace="10px" src="http://img.vz.ru/upimg/ta_/ta_noimage.jpg" align="left" style="border: none;"/>\n\t\t\t\n\t\t\tРоснефть добилась ареста акций АФК «Система» из-за «публичной позиции корпорации», заявил официальный представитель нефтяной компании Михаил Леонтьев. По его утверждению, позиция АФК &amp;laquo;построена на абсолютном игнорировании и презрении к легальной правовой системе российского государства&amp;raquo;, передает &lt;a href=&quot;https://ria.ru/&quot; target=&quot;_blank&quot;&gt;РИА &amp;laquo;Новости&amp;raquo;&lt;/a&gt;. \nРанее стало известно, что на 31,76% акций МТС, 100% акций &amp;laquo;Медси&amp;raquo; и 90,47% акций БЭСК, принадлежащих АФК &amp;laquo;Система&amp;raquo;, &lt;a href=&quot;https://vz.ru/news/2017/6/26/876145.html&quot; target=&quot;_blank&quot;&gt;был наложен арест&lt;/a&gt; в качестве обеспечительных мер на основании решения Арбитражного суда Башкирии по иску Роснефти, Башнефти и правительства Башкортостана. По оценкам АФК, из-

In [0]:
cross_val('sentiment', 'f1', 5)

In [0]:
model = ModelSent(alpha=0.09187)
model.fit(train_frame['title_proc'], train_frame['sentiment'])

In [0]:
preds = model.predict(frame['title_proc'])
#sum(preds == frame.sentiment[-200:]) / len(preds)

In [0]:
frame['sentiment'] = preds

In [0]:
frame.cat = [cat_map[i] for i in frame.cat]
frame.sample(frac=1).head(200).to_csv(path_dir + 'test_sample.csv')

In [0]:
frame.head()

,u_id,title,description,title_proc,cat,sentiment
0,1074389,&quot;Газпром&quot; задумался о поставках газа...,&quot;Газпром&quot; обсуждает поставки газа в ...,газпром задумался о поставках газа в индию газ...,Другое,0
1,1635915,Замгендиректора дочерней компании &quot;Газпро...,"По версии следствия, в 2015 году злоумышленник...",замгендиректора дочерней компании газпром нефт...,Другое,-1
2,1117120,Украина против «Газпрома»: покупаем попкорн,Власти Украины арестовали уставной капитал доч...,украина против газпрома покупаем попкорн власт...,Другое,-1
3,20405,Белоусов предложил Сечину определиться с форма...,"Помощник президента России, председатель совет...",белоусов предложил сечину определиться с форма...,Другое,1
4,1010629,Суд удовлетворил ходатайство «Роснефти» об уве...,&lt;p&gt;Арбитражный суд Республики Башкортост...,суд удовлетворил ходатайство роснефти об увели...,Другое,0


In [0]:
frame['predictions'] = [-1 for i in frame['title_proc'][:-200]] + list(preds)

In [0]:
analysis = frame[-200:][frame['predictions'] != frame.sentiment]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [0]:
analysis[analysis['sentiment'] == 0]

,title,description,cat,sentiment,title_proc,predictions
1215,Глава совета директоров Eni объяснила приостан...,"Она обусловлена санкциями, но итальянская комп...",4,0,глава совета директоров eni объяснила приостан...,-1
752,Момент убийства топ-менеджера подрядчика «Луко...,"Следственный комитет опубликовал видео, на кот...",5,0,момент убийства топ менеджера подрядчика лукой...,-1
1019,«Нафтогаз» отреагировал на снятие ареста с акц...,"<img alt=""Preview"" align=""left"" style=""margin-...",5,0,нафтогаз отреагировал на снятие ареста с акций...,1
823,«Нафтогаз» заявил о готовности обсуждать отказ...,"Витренко добавил, что соответствующий иск уже ...",5,0,нафтогаз заявил о готовности обсуждать отказ о...,-1
402,Докачать до 200 млрд: рекорд «Газпрому» мешает...,"За 11,5 месяцев «Газпром» экспортировал 188 мл...",5,0,докачать до 200 млрд рекорд газпрому мешает по...,1
583,Для газовых турбин нашли место на Дону // «Сил...,"Как выяснил “Ъ”, попавшие под санкции США «Сил...",5,0,для газовых турбин нашли место на дону силовые...,1
702,Глава «Газпрома» обсудил с и. о. вице-премьера...,"<img alt=""Preview"" align=""left"" style=""margin-...",5,0,глава газпрома обсудил с и о вице премьера арм...,-1
105,ФАС может до конца года рассмотреть дело по жа...,Глава службы Игорь Артемьев может попросить пр...,5,0,фас может до конца года рассмотреть дело по жа...,-1
1230,&quot;Газпром нефть&quot; пересмотрит размер ф...,Глава &quot;Газпром нефти&quot; Александр Дюко...,3,0,пересмотрит размер финансирования фк если появ...,1
218,«Нафтогаз» обратился в суд США из-за спора с «...,,5,0,нафтогаз обратился в суд сша из за спора с газ...,-1


In [0]:
frame.loc[324]['title']

'«Газпром» и OMV обменяются активами'

**10 most simalar news**

In [0]:
path_dir = '/content/drive/My Drive/Work/OnlineNewsScoring/data/'

In [0]:
frame = pd.read_csv(path_dir + 'neftegaz_full.csv')
frame = frame.fillna('')
frame.head()

,u_id,title,description
0,299009,"""Амурский ГПЗ"" ""Газпрома"" может построить ""доч...",Завод мощностью 42 млрд куб. м газа в год буде...
1,172020,"""Башнефть"" предоставит ""Роснефти"" заем на 50 м...","<p>Башкирская дочка «Роснефти», вошедшая в ее ..."
2,20327,"""Башнефть"" примет оферту ""Роснефти""","Совет директоров ""Башнефти"" рекомендовал акцио..."
3,373771,"""Башнефть"": примерно 78% миноритарных акционер...",
4,274562,"""Белогорье"" на выезде обыграло ""Газпром-Югра"" ...",<p>Волейболисты белгородского «Белогорья» одер...


In [0]:
frame.iloc[1]['title']

'"Башнефть" предоставит "Роснефти" заем на 50 млрд рублей'

In [0]:
!pip install deeppavlov

In [0]:
!python3 -m deeppavlov install squad_bert

2020-05-09 00:08:03.960 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_bert' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/squad/squad_bert.json'
     |████████████████████████████████| 110.5MB 36kB/s 
     |████████████████████████████████| 512kB 42.8MB/s 
     |████████████████████████████████| 3.8MB 41.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=9ff1d481082b3cf671bcc12bb61aab4e68aeac8e26bc03d1a0ab4ba85be99ac8
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.

In [0]:
!python -m deeppavlov install gobot_dstc2_best

2020-05-17 12:28:21.286 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'gobot_dstc2_best' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/go_bot/gobot_dstc2_best.json'
     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2333084 sha256=b70dfa4301b310718bb1c9da9a7a3560776cffb9c4df51ec0eb5bf5303c9ae14
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext
  Found existing installation: fasttext 0.8.22
    Uninstalling fasttext-0.8.22:
      Successfully uninstalled fasttext-0.8.22
     |████████████████████████████████| 10.4MB 2.5MB/s 
     |████████████████████████████████| 2.2MB 48.2MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Suc

In [0]:
from deeppavlov import configs, build_model
from gensim.models import FastText

ner_model = build_model(configs.ner.ner_rus, download=True)

2020-05-17 12:37:53.204 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_rus_v3_cpu_compatible.tar.gz download because of matching hashes
2020-05-17 12:38:24.407 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/embeddings/lenta_lower_100.bin download because of matching hashes
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2020-05-17 12:38:25.948 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vo








Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use `tf.cast` instead.



2020-05-17 12:38:32.898 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 756: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


2020-05-17 12:38:34.583 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 756: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Instructions for updating:
Use standard file APIs to check for files with this prefix.


2020-05-17 12:38:36.485 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_rus/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_rus/model


In [0]:
ind = 3
print(frame['title'][ind])
print(ner_model([frame['title'][ind]])[0][0])
print(ner_model([frame['title'][ind]])[1][0])

"Башнефть": примерно 78% миноритарных акционеров приняли предложение "Роснефти", - Аналитики Sberbank Investment Research
['``', 'Башнефть', "''", ':', 'примерно', '78', '%', 'миноритарных', 'акционеров', 'приняли', 'предложение', '``', 'Роснефти', "''", ',', '-', 'Аналитики', 'Sberbank', 'Investment', 'Research']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG']


In [0]:
ner_res = ner_model(frame['title'][:100])
for i, (tokens, classes) in enumerate(zip(ner_res[0], ner_res[1])):
    print(tokens)
    print(classes)

['``', 'Амурский', 'ГПЗ', "''", '``', 'Газпрома', "''", 'может', 'построить', '``', 'дочка', "''", 'CNPC']
['O', 'O', 'B-ORG', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG']
['``', 'Башнефть', "''", 'предоставит', '``', 'Роснефти', "''", 'заем', 'на', '50', 'млрд', 'рублей']
['O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O']
['``', 'Башнефть', "''", 'примет', 'оферту', '``', 'Роснефти', "''"]
['O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O']
['``', 'Башнефть', "''", ':', 'примерно', '78', '%', 'миноритарных', 'акционеров', 'приняли', 'предложение', '``', 'Роснефти', "''", ',', '-', 'Аналитики', 'Sberbank', 'Investment', 'Research']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG']
['``', 'Белогорье', "''", 'на', 'выезде', 'обыграло', '``', 'Газпром-Югра', "''", 'в', 'ЧР', 'по', 'волейболу']
['O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'B-LOC', 'O', 'O']
['``', 'Белогорье', "''", 'по

In [0]:
title_ners = {}
descr_ners = {}
counter = 0
for i, row in tqdm(frame.iterrows()):
    title_res = ner_model([row['title']])
    title_ners[row['u_id']] = (title_res[0][0], title_res[1][0])
    if row['description'] != '':
        try:
            descr_res = ner_model([row['description']])
            descr_ners[row['u_id']] = (descr_res[0][0], descr_res[1][0])
        except:
            print('Exceeded: ', row['u_id'])
    counter += 1
    if counter % 1000 == 0:
        with open(path_dir + 'ner_title_maps.pkl', 'wb') as file:
            pkl.dump(title_ners, file)
        with open(path_dir + 'ner_descr_maps.pkl', 'wb') as file:
            pkl.dump(descr_ners, file)

In [0]:
with open(path_dir + 'ner_title_maps.pkl', 'rb') as file:
    title_maps = pkl.load(file)
with open(path_dir + 'ner_descr_maps.pkl', 'rb') as file:
    descr_maps = pkl.load(file)

In [0]:
title_maps

{299009: (['"',
   'Амурский',
   'ГПЗ',
   '"',
   '"',
   'Газпрома',
   '"',
   'может',
   'построить',
   '"',
   'дочка',
   '"',
   'CNPC'],
  ['O',
   'B-ORG',
   'I-ORG',
   'O',
   'O',
   'B-ORG',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-ORG']),
 172020: (['"',
   'Башнефть',
   '"',
   'предоставит',
   '"',
   'Роснефти',
   '"',
   'заем',
   'на',
   '50',
   'млрд',
   'рублей'],
  ['O', 'B-ORG', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O']),
 20327: (['"', 'Башнефть', '"', 'примет', 'оферту', '"', 'Роснефти', '"'],
  ['O', 'B-ORG', 'O', 'O', 'O', 'O', 'B-ORG', 'O']),
 373771: (['"',
   'Башнефть',
   '"',
   ':',
   'примерно',
   '78',
   '%',
   'миноритарных',
   'акционеров',
   'приняли',
   'предложение',
   '"',
   'Роснефти',
   '"',
   ',',
   '-',
   'Аналитики',
   'Sberbank',
   'Investment',
   'Research'],
  ['O',
   'B-ORG',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-ORG',
   'O',
   'O',
 

In [0]:
ner = set()
for tokens, classes in title_maps.values():
    for token, cl in zip(tokens, classes):
        if 'ORG' in cl:
            ner.add(token)

for tokens, classes in descr_maps.values():
    for token, cl in zip(tokens, classes):
        if 'ORG' in cl:
            ner.add(token)

In [0]:
ner

{'Росгидромета',
 'Тюмень',
 'Батьковщина',
 'АНКОР',
 'Синара',
 'Пулково',
 'СРП',
 'Fortum',
 'газеты',
 'движения',
 'Atlantic',
 'народной',
 'Тверь',
 'Glencor',
 'Пошехонский',
 'Journal',
 'Горный',
 'NPC',
 'структур',
 'Гринхаус',
 'Форум',
 'Арбитражном',
 'Согаза',
 'НОВАТЭКу',
 'Енисея',
 'Комиссии',
 'ПАЗ',
 'АЛРОСы',
 'ROSNEFT',
 'Солид',
 'Ассоциацией',
 'РОСНЕФТИ',
 'Дождю',
 'Генеральную',
 'Стройгазмонтаж',
 'Польская',
 'кладбище',
 'Сада',
 'Enjoy',
 'Сибирская',
 'Trumpet',
 'ИЭ',
 'Абсолют',
 'Schlumberger',
 'Уралхиму',
 'p',
 'Севмаш',
 'Газпроминвестхолдинге',
 'Кайрату',
 'Академией',
 'Дальневосточным',
 'ОП',
 'свободы',
 'Квирис',
 'Космических',
 'зеленые',
 'Известий',
 'Сухой',
 'Псковэнергосбыта',
 'Высоцк',
 'Linde',
 'Itochu',
 'Мегафона',
 'Iraq',
 'студии',
 'баррикадам',
 'регион',
 'National',
 'Мосгорсуде',
 'товарно',
 'национальное',
 'Русгаздобычей',
 'Конгресса',
 'QIA',
 'Госкомпания',
 'СГК',
 'NRC',
 'Eni',
 'Уватнефтегаз',
 'и',
 'эконом

In [0]:
def extract_one_new(news_ner):
    keys = np.array(news_ner[0])[np.array(news_ner[1]) != 'O']
    indexes = []
    for i, mark in enumerate(news_ner[1]):
        if mark != 'O': 
            continue
        clear = re.sub(r'[\W]', '', news_ner[0][i])
        if clear != '' and len(clear) > 1:
            indexes.append(i)
    appendix = np.array(news_ner[0])[indexes]
    return (keys, ' '.join(appendix))

def extract_features(maps):
    result = {}
    for k, v in maps.items():
        result[k] = extract_one_new(v)
    return result

title_maps_match = extract_features(title_maps)
descr_maps_match = extract_features(descr_maps)

unimap = {}
for k, v in title_maps_match.items():
    if k in descr_maps_match:
        descr_val = descr_maps_match[k]
        unimap[k] = (np.union1d(v[0], descr_val[0]), v[1], descr_val[1])
    else:
        unimap[k] = v

Необходимо собрать словари для оптимизации

In [0]:
entity_dict = {}
for k, v in unimap.items():
    for ent in v[0]:
        if ent in entity_dict:
            entity_dict[ent].add(k)
        else:
            entity_dict[ent] = set([k])

In [0]:
id_news_map = {}
new_ids_map = {}
for i, row in frame.set_index('u_id').iterrows():
    id_news_map[i] = row['title']

for k, v in unimap.items():
    new_ids_map[v[1]] = k

In [0]:
fasttext = FastText(size=360, window=3, min_count=1, iter=100, workers=-1, min_n=1, max_n=5)

In [0]:
name_corpus = np.array([[v[1]] for k, v in unimap.items()])
print(name_corpus[:10])
fasttext.build_vocab(name_corpus)
corp_count = fasttext.corpus_count
n_iter = 100
print(sum([1 for _ in name_corpus]))
fasttext.train(name_corpus, total_examples=corp_count, epochs=n_iter)

[['может построить дочка']
 ['предоставит заем на 50 млрд рублей']
 ['примет оферту']
 ['примерно 78 миноритарных акционеров приняли предложение Аналитики']
 ['на выезде обыграло по волейболу']
 ['победило матче Финала шести Кубка']
 ['должна не предлагать платить за российский газ']
 ['рабочий полдень']
 ['государство может лишиться большинства совете директоров']
 ['от льгот для может зависеть вся налоговая реформа отрасли']]
20000


In [0]:
def find_match(unmap, id_news_map, new_ids_map, entity_dict, ner_model, new, top_n=5):
    new = new.replace('&quot;', '"')
    ner = ner_model([new])
    print(new)
    features, appendix = extract_one_new(ner)
    scores = dict([(i, 0.) for i in unmap.keys()])
    
    for f in features:
        if f not in entity_dict:
            continue
        for ident in entity_dict[f]:
            scores[ident] += 1
            
    most_sim = fasttext.most_similar(new, topn=fasttext.corpus_count)
    ids = [new_ids_map[i[0]] for i in most_sim]
    for ident, (name, score) in zip(ids, most_sim):
        scores[ident] += score
        
    scores = sorted([(id_news_map[i[0]], i[1] / (len(features) + 1)) for i in scores.items()], key=lambda x: -x[1])
    return scores[:top_n], features

In [0]:
#id = np.random.randint(1, 20000, 1)[0]
top, ne = find_match(unimap, id_news_map, new_ids_map, entity_dict, ner_model, frame.iloc[id]['title'], 6)

Источник: "Роснефть" продолжает проверку законности решений АФК "Система" по "Башнефти"


In [0]:
news = []
scores = []
for new, score in top:
    news.append(new)
    scores.append(score)
    print('Title: ' + new)
    print('Score: ' + str(score))
top_frame = pd.DataFrame({'new': news, 'score': scores})

Title: Накал интриги: суд выносит первые решения против "Системы"
Score: 0.9349761520113263
Title: "Роснефть" оценила потери "Башнефти" от действий "Системы" в 180 млрд руб в ценах тех лет
Score: 0.931651098387582
Title: "Роснефть" может обжаловать решение суда по иску к АФК "Система" после изучения документов
Score: 0.9301510453224182
Title: Башкирский арбитраж объяснил свои мотивы: "Система" выводила активы, зная, что владеет "Башнефтью" незаконно
Score: 0.9280856677464077
Title: Суд частично удовлетворил иск "Роснефти" к АФК "Система", НК довольна решением
Score: 0.9257384879248483
Title: "Роснефть" подала в суд на АФК "Система" из-за обесценения "Башнефти" на 4 млрд рублей
Score: 0.9246908043112073


In [0]:
top_frame

,new,score
0,«Амурский ГПЗ» «Газпрома» может построить «доч...,0.945220
1,Китайская CNPC поставит оборудование на Амурск...,0.881736
2,"""Амурский ГПЗ"" ""Газпрома"" может построить ""доч...",0.800000
3,Газпром: В строительстве Амурского ГПЗ примет ...,0.715628
4,Путин дал старт строительству первого фундамен...,0.710025


In [0]:
frame.iloc[id]

u_id                                                      554861
title               Лукойл продаст треть своих заправок в России
description    ЛУКОЙЛ может продать около 700 своих автозапра...
title_proc     лукойл продаст треть своих заправок в россии л...
cat                                                            5
sentiment                                                      0
Name: 1036, dtype: object

In [0]:
find_match(unimap, id_news_map, entity_dict, ner_model, frame.iloc[0]['title'])

[[['"', 'Амурский', 'ГПЗ', '"', '"', 'Газпрома', '"', 'может', 'построить', '"', 'дочка', '"', 'CNPC']], [['O', 'B-ORG', 'I-ORG', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG']]]


[('"Амурский ГПЗ" "Газпрома" может построить "дочка" CNPC', 1.0),
 ('«Амурский ГПЗ» «Газпрома» может построить «дочка» CNPC', 1.0),
 ('Китайская CNPC поставит оборудование на Амурский ГПЗ', 1.0),
 ('Газпром: В строительстве Амурского ГПЗ примет участие китайская компания',
  0.75),
 ('Китайский партнер "Газпрома" в возведении Амурского ГПЗ. Главное', 0.75)]

In [0]:
!pip install git+https://github.com/IINemo/active_learning_toolbox

  Cloning https://github.com/IINemo/active_learning_toolbox to /tmp/pip-req-build-nutg4hvn
  Running command git clone -q https://github.com/IINemo/active_learning_toolbox /tmp/pip-req-build-nutg4hvn
     |████████████████████████████████| 645kB 2.8MB/s 
  Created wheel for actleto: filename=actleto-0.1.0-cp36-none-any.whl size=15392 sha256=b7665d14bd931cbc7cccd20edaaeebe62870b4f5e2e350f59d1b78187a90256f
  Stored in directory: /tmp/pip-ephem-wheel-cache-m9u76var/wheels/fd/99/47/a1b304c9554210ecff1a7d0e8dc4101a4fb48142bab09670a0
  Created wheel for annoy: filename=annoy-1.16.0-cp36-cp36m-linux_x86_64.whl size=309750 sha256=cf4c37b57aafced6a5e77f8bb8fb78331f0de6c96de7220cec681f89049586df
  Stored in directory: /root/.cache/pip/wheels/39/34/9c/8d8eb3abce9375bade7b42bca387700b5f3aa18414d27272f0
Successfully built actleto annoy


# Active Learning test

In [0]:
!pip install modAL

In [0]:
from modAL.models import ActiveLearner
import pandas as pd
import numpy as np
import re
from tqdm import tqdm_notebook as tqdm
import random
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from google.colab import drive
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.naive_bayes import *
import warnings
import pickle as pkl
#from langdetect import detect
warnings.filterwarnings('ignore')

In [0]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
train_frame = pd.read_excel('/content/drive/My Drive/Work/OnlineNewsScoring/data/dataset_news_2018_2019_marked_full.xlsx')
train_frame = train_frame[[i for i in train_frame.columns if 'Unnamed' not in i]]
train_frame = train_frame[['title', 'description', 'cat', 'sentiment']]
train_frame = train_frame.fillna('')
train_frame = train_frame[np.logical_or(train_frame['title'] != '', train_frame['description'] != '')]
train_frame['cat'] = train_frame['cat'].astype('int')
train_frame['sentiment'] = train_frame['sentiment'].astype('int')
train_frame['sentiment'] = train_frame['sentiment'].replace({9: 0})

In [0]:
def condition(label):
    conds = []
    conds.append('o' != label.lower())
    return np.any(conds)
  
def condition_str(word):
    conds = []
    conds.append(word != '')
    conds.append('&quot;' not in word)
    conds.append('"' != word)
    conds.append("'" != word)
    return np.all(conds)
  
def create_feature_col(frame, use_ner=False):
    frame_copy = frame.sample(frac=1).reset_index(drop=True)
    titles = []
    for title in tqdm(frame_copy['title'] + ' ' + frame_copy['description']):
        title = ' '.join([i for i in title.split(' ') if '$' not in i])
        title = title.replace('&quot;', '"')
        if use_ner:
            mapping = ner_model([title])
            sent = np.array(mapping[0][0])
            ner = mapping[1][0]
            new_sent_ind = []
            for i, label in enumerate(ner):
                if not condition(label):
                    new_sent_ind.append(i)
            string = ' '.join(sent[new_sent_ind])
        else:
            string = title
        string = string.lower()
        string = ' '.join([i for i in string.split() if condition_str(i)])
        string = re.sub(r'&amp;laquo;', '', string)
        string = re.sub(r'<.*>', '', string)
        string = re.sub(r'[\W]', ' ', string)
        string = re.sub(r' +', ' ', string)
        titles.append(string.strip())
    frame_copy['title_proc'] = titles
    return frame_copy

In [0]:
train_frame = create_feature_col(train_frame)

In [0]:
class ModelCat:
  
    def __init__(self, alpha):
        self.vect = TfidfVectorizer(min_df=10, max_df=500)
        self.model = MultinomialNB(alpha)
    
    def fit(self, X, y):
        data = self.vect.fit_transform(X)
        self.model.fit(data, y)
        
    def predict(self, X):
        data = self.vect.transform(X)
        preds = self.model.predict(data)
        for i, x in enumerate(X):
            if x.split(' ')[0].lower() == 'приказ':
                preds[i] = 5
                continue
            if x.split(' ')[0].lower() == 'постановление':
                preds[i] = 6
                continue
            for part in x.split(' '):
                if 'инвест' in part.lower():
                    preds[i] = 3
                    break
                if 'санкц' in part.lower() and 'несанкц' not in part.lower():
                    preds[i] = 4
                    break
                if 'развивать' == part.lower():
                    preds[i] = 2
                    break
                if part.lower() in ['убытки', 'потери', 'штраф']:
                    preds[i] = 1
        return preds

In [0]:
class ModelSent:
    
    def __init__(self, alpha):
        self.vect = TfidfVectorizer(min_df=10, max_df=500)
        self.model = MultinomialNB(alpha)
    
    def fit(self, X, y):
        data = self.vect.fit_transform(X)
        self.model.fit(data, y)
        
    def predict(self, X):
        data = self.vect.transform(X)
        preds = self.model.predict(data)
        for i, x in enumerate(X):
            for part in x.split(' '):
                if 'обсуд' in part:
                    preds[i] = 0
                    break
        return preds

In [0]:
class PseudoLabelTrainer:

    def __init__(self, model, upper_threshold, lower_threshold):
        self.model = model
        self.upper_threshold = upper_threshold
        self.lower_threshold = lower_threshold

    def fit(self, labeled_X, labeled_Y, unlabeled_X, sample_rate=0.1):
        sample_size = len(unlabeled_X) * sample_rate
        while len(unlabeled_index) > 0:

            self.model.fit(labeled_X, labeled_Y)
            
            data = unlabeled_X[:sample_size]
            unlabeled_X = unlabeled_X[sample_size:]

            probs = self.model.predict_proba(data)

            max_probs = np.max(probs, axis=1)
            selected_indexes = (max_probs > self.upper_threshold) | (max_probs < self.lower_threshold)

            selected_labels = np.argmax(probs, axis=1)[selected_indexes]
            selected_rows = data[selected_indexes]

            labeled_X = np.vstack((labeled_X, selected_rows))
            labeled_Y = np.vstack((labeled_Y, selected_labels))
        return self.model

array([ True,  True,  True, False])